In [1]:
#default_exp hypster_prepare

In [2]:
#export
from hypster.oo_hp import *

In [3]:
#export
from inspect import signature
import functools
from collections import OrderedDict

In [4]:
#export
class HypsterPrepare(HypsterBase):
    def __init__(self, call, base_call, *args, **kwargs):
        self.call            = call
        self.base_call       = base_call
        self.args            = args
        self.kwargs          = kwargs
        self.trials_sampled  = set()
        self.studies_sampled = set()
        self.base_object     = None
        self.result          = None

    def sample(self, trial):
        if trial.study.study_name not in self.studies_sampled:
            self.trials_sampled = set()
        elif trial.number in self.trials_sampled:
            return self.result           
        
        if self.base_call is not None:
            self.base_object = self.base_call.sample(trial)

        #TODO: add HpToggle Here
        #self.sampled_args   = [sample_hp(arg, trial) for arg in self.args]
        #sampled_kwargs      = [sample_hp(arg, trial) for arg in self.kwargs.values()]
        #self.sampled_kwargs = OrderedDict(zip(self.kwargs.keys(), sampled_kwargs))
        
        self.sampled_args   = populate_iterable(self.args, trial)
        self.sampled_kwargs = populate_dict(self.kwargs, trial)
        
        self.trials_sampled.add(trial.number)
        self.studies_sampled.add(trial.study.study_name)

        if self.base_object:
            if len(self.sampled_args) == 0 and len(self.sampled_kwargs) == 0:
                self.result = getattr(self.base_object, self.call)
            else:
                self.result = getattr(self.base_object, self.call)(*self.sampled_args, **self.sampled_kwargs)
        else:
            self.result = self.call(*self.sampled_args, **self.sampled_kwargs)
        return self.result
    
    def __call__(self, *args, **kwargs):
        #print(f"args {args}, kwargs {kwargs}")
        self.args = args
        self.kwargs = kwargs
        return self
    
    def __getattr__(self, name, *args, **kwargs):
        #print(f"name {name}, args {args}, kwargs {kwargs}")
        return HypsterPrepare(name, self, *args, **kwargs)

In [5]:
#export
def prepare(call):
    @functools.wraps(call)
    def wrapper_decorator(*args, **kwargs):
        #print(f"args: {args}")
        #print(f"kwargs: {kwargs}")
        all_args = list(args) + list(kwargs.values())
        if any([contains_hypster(arg, HYPSTER_TYPES) for arg in all_args]):
            return HypsterPrepare(call, None, *args, **kwargs)
        else:
            return call(*args, **kwargs)
    return wrapper_decorator

# Test Prepare

In [6]:
@prepare
def foo(a, *args, b="hi!", c=None, **kwargs):
    if c is not None:
        print(f"{a} and {b} and {c}")
    else:
        print(f"{a} and {b}")
        
    for arg in args:
        print(arg)
    
    return f"returned {a} and {b} and {c}"

In [7]:
@prepare
class Cls:
    def __init__(self, a, *args, b="hi!", c=None, **kwargs):
        if c is not None:
            print(f"{a} and {b} and {c}")
        else:
            print(f"{a} and {b}")

        for arg in args:
            print(arg)

        #return f"returned {a} and {b} and {c}"
    
    def shmul(self, batch):
        print(batch)

In [8]:
hps = foo("hi", "hello", "hola", b="shmuli")

hi and shmuli
hello
hola


In [9]:
hps = Cls("hi", "hello", "hola", b=HpCategorical("b", ["Shmuli", "Buli"]))

In [10]:
z = hps.shmul(batch=32)

In [11]:
#export
import optuna

In [12]:
#export
def run_func_test(x, n_trials=5):
    def objective(trial):
        y = x.sample(trial)
        print(y)
        return 1.0

    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    study.optimize(objective, n_trials=n_trials, timeout=600)

In [13]:
run_func_test(z)

hi and Buli
hello
hola
32
None
hi and Buli
hello
hola
32
None
hi and Buli
hello
hola
32
None
hi and Buli
hello
hola
32
None
hi and Buli
hello
hola
32
None


In [14]:
class Cls():
    def __init__(self, name, last_name="", nickname=""):
        self.name = name
        self.last_name = last_name
        self.nickname = nickname
        print(f"{self.name} {self.last_name} {self.nickname}")  

In [15]:
c = Cls("Gilad", nickname="The King!")

Gilad  The King!


In [16]:
Cls2 = prepare(Cls)

In [17]:
x = Cls2("Gilad", HpCategorical("last", ["The King!", "The Best King!"]))

In [18]:
run_func_test(x)

Gilad The Best King! 
Gilad The King! 
Gilad The Best King! 
Gilad The Best King! 
Gilad The King! 


## HpToggle

In [19]:
Cls2 = prepare(Cls)

In [20]:
x = Cls2("Gilad", last_name=HpToggle("toggle_last", HpCategorical("last", ["The King!", "The Best King!"])))

In [21]:
run_func_test(x)

Gilad  
Gilad The King! 
Gilad  
Gilad  
Gilad  


In [22]:
from nbdev.export import notebook2script

In [23]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_api.ipynb.
Converted 02_oo_hp.ipynb.
Converted 03_hypster_prepare.ipynb.
Converted 04_tabular_api.ipynb.
Converted 05_sklearn.ipynb.
Converted fastai_adult_tutorial.ipynb.
Converted index.ipynb.
